In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import string
nltk.download('words')

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import word2vec

from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import word2vec


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [42]:
!pip install gensim
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.23
    Uninstalling Cython-0.29.23:
      Successfully uninstalled Cython-0.29.23


In [3]:
df=pd.read_csv("C:\\Users\\hp\\Documents\\MGP\\CyberBullying Detection\\Dataset\\twitter_parsed_dataset.csv")
df.head()

,index,id,Text,Annotation,oh_label
0,5.74948705591165E+017,5.74948705591165E+017,@halalflaws @biebervalue @greenlinerzjm I read...,none,0.0
1,5.71917888690393E+017,5.71917888690393E+017,@ShreyaBafna3 Now you idiots claim that people...,none,0.0
2,3.90255841338601E+017,3.90255841338601E+017,"RT @Mooseoftorment Call me sexist, but when I ...",sexism,1.0
3,5.68208850655916E+017,5.68208850655916E+017,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",racism,1.0
4,5.75596338802373E+017,5.75596338802373E+017,#mkr No No No No No No,none,0.0


In [4]:
df["Text"].sample(1).values[0]

"@VeraVanHorne @RusConCan @VICE Sorry, is it Olga, or Tanya, or Lena, or Marina. How's you little Putin sweat shop? http://t.co/stp5NmjZRY"

In [5]:
sent="Go to the url http:\\helloabdullaj\\google.com and open it"
sent=re.sub(r"\S*https?:\S*","|url|",sent)
print(sent)

Go to the url |url| and open it


In [6]:
def change_case_lower(text):
    sample = text
    if(type(sample)==str):
        sample = " ".join([x.lower() for x in sample.split()])
    
    return sample

def remove_url(text):
    sample = text
    sample = re.sub(r"\S*https?:\S*", '', sample) #links and urls
    
    return sample

def remove_html(text):
    sample = text
    comp = re.compile(r'<.*?>')
    sample = re.sub(comp, '', sample)
    
    return sample

def remove_punctuations(text):
    sample = text
    sample = re.sub('\[.*?\]', '', sample) #text between [square brackets]
    sample = re.sub('\(.*?\)', '', sample) #text between (parenthesis)
    sample = re.sub('[%s]' % re.escape(string.punctuation), '', sample) #punctuations
    sample = re.sub("[''""...“”‘’…]", '', sample) #list of quotation marks
    
    return sample

only_english = set(nltk.corpus.words.words())
def remove_special_characters(text):
    sample = text
    sample = re.sub(r'\n', ' ', sample) #new line character
    sample = re.sub(r'\\n', ' ', sample) #new line character
    sample = ' '.join([w for w in nltk.wordpunct_tokenize(sample) if w.lower() in only_english or not w.isalpha()]) #doesn't remove indian languages
    sample = ' '.join(list(filter(lambda ele: re.search("[a-zA-Z\s]+", ele) is not None, sample.split()))) #languages other than english
    sample = " ".join([x.strip() for x in sample.split()])
    
    return sample

def remove_hashtags_total(text):
    sample = text
    sample = re.sub('#', ' ', sample)
    
    return sample

def remove_hashtags_only(text):
    sample = text
    sample = ' '.join([x for x in s.split() if not x.startswith('#')])
    
    return sample

def remove_emojis(text):
    sample = text
    sample = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE).sub(r'', sample) #emojis and symbols
    return sample

In [7]:
s = """@DrBiden Ask 83 Joe about his thoughts <b> on packing </b> the Supreme Court. 😅😄 He seem to have any after 50 years in politics. #JoeBiden ...more sinister" \\n\\n#Covid19UK \\n#Covid #News \\n\\nJust enter "empty testing stations" СВОИМИ РУКАМИ into your search engIne and https://t.co/omPuIJzVnl"""


In [8]:
change_case_lower(s)

'@drbiden ask 83 joe about his thoughts <b> on packing </b> the supreme court. 😅😄 he seem to have any after 50 years in politics. #joebiden ...more sinister" \\n\\n#covid19uk \\n#covid #news \\n\\njust enter "empty testing stations" своими руками into your search engine and https://t.co/ompuijzvnl'

In [9]:
remove_emojis(s)

'@DrBiden Ask 83 Joe about his thoughts <b> on packing </b> the Supreme Court.  He seem to have any after 50 years in politics. #JoeBiden ...more sinister" \\n\\n#Covid19UK \\n#Covid #News \\n\\nJust enter "empty testing stations" СВОИМИ РУКАМИ into your search engIne and https://t.co/omPuIJzVnl'

In [10]:
remove_url(s)

'@DrBiden Ask 83 Joe about his thoughts <b> on packing </b> the Supreme Court. 😅😄 He seem to have any after 50 years in politics. #JoeBiden ...more sinister" \\n\\n#Covid19UK \\n#Covid #News \\n\\nJust enter "empty testing stations" СВОИМИ РУКАМИ into your search engIne and '

In [11]:
remove_hashtags_only(s)

'@DrBiden Ask 83 Joe about his thoughts <b> on packing </b> the Supreme Court. 😅😄 He seem to have any after 50 years in politics. ...more sinister" \\n\\n#Covid19UK \\n#Covid \\n\\nJust enter "empty testing stations" СВОИМИ РУКАМИ into your search engIne and https://t.co/omPuIJzVnl'

In [12]:
def clean_data(text):
    sample=text
    sample=change_case_lower(sample)
    sample=remove_url(sample)
    sample=remove_html(sample)
    sample=remove_punctuations(sample)
    sample=remove_special_characters(sample)
    sample=remove_emojis(sample)
    sample=remove_hashtags_total(sample)
    return sample

In [13]:
clean_data(s)

'ask joe about his on the supreme court he seem to have any after in politics more sinister nncovid19uk news enter empty testing into your search engine and'

In [14]:
df["Text"]=df["Text"].apply(lambda x: clean_data(x) if(type(x)==str) else x )

In [15]:
df.head()

,index,id,Text,Annotation,oh_label
0,5.74948705591165E+017,5.74948705591165E+017,i read them in change in meaning the history o...,none,0.0
1,5.71917888690393E+017,5.71917888690393E+017,shreyabafna3 now you claim that people who tri...,none,0.0
2,3.90255841338601E+017,3.90255841338601E+017,call me but when i go to an auto place id rath...,sexism,1.0
3,5.68208850655916E+017,5.68208850655916E+017,g0ssipsquirrelx wrong the example of and the e...,racism,1.0
4,5.75596338802373E+017,5.75596338802373E+017,no no no no no no,none,0.0


In [16]:
len(only_english)

235892

In [17]:
stops = set(stopwords.words('english'))
def remove_stop_words(text, cores = 2):
    
    sample = text
    sample = sample.lower()
    sample = [word for word in sample.split() if not word in stops]
    sample = ' '.join(sample)
    
    return sample
def get_wordnet_pos(word):
    
    treebank_tag = nltk.pos_tag([word])[0][1]
    
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

lemmatizer=WordNetLemmatizer()
def lemma_clean_text(text, cores = 1):
 
    sample = text
    sample = sample.split(' ')
    sample = [lemmatizer.lemmatize(word.lower(), get_wordnet_pos(word.lower())) for word in sample if(len(word)>0)]
    sample =  ' '.join(sample)
    
    return sample
stem=PorterStemmer()
def stem_clean_text(text):
 
    sample = text
    sample = sample.split()
    sample = [stem.stem(word) for word in sample]
    sample = ' '.join(sample)
    
    return sample
def correct_spelling(text):
    
    sample = text
    sample = str(TextBlob(text).correct())
    
    return sample

In [18]:
s="RT @WittySam: Colin's review of Kat &amp; Andre is equivalent of a teacher saying well done on writing your name in pencil. #mkr"

In [19]:
s=remove_stop_words(s)
print(s)
lemma_clean_text(s)

rt @wittysam: colin's review kat &amp; andre equivalent teacher saying well done writing name pencil. #mkr


"rt @wittysam: colin's review kat &amp; andre equivalent teacher say well do write name pencil. #mkr"

In [20]:
def correct_text(text,stem=False,lemma=False,spell=False):
    if lemma and stem:
        raise Exception('Either stem or lemma can be true, not both!')
        return text
    sample=text
    sample=remove_stop_words(sample)
    if(stem):
        sample=stem_clean_text(sample)
    if(lemma):
        sample=lemma_clean_text(sample)
    if(spell):
        sample=correct_spelling(sample)
    return sample


In [21]:
correct_text(s,lemma=True,spell=True)

"it @wittysam: coin's review at &amp; andre equivalent teacher say well do write name pencil. #mr"

In [22]:
df['correct_text'] = df['Text'].apply(lambda x: correct_text(x,stem=True,spell=True) if (type(x)==str) else x)


In [24]:
df.to_csv("C:\\Users\\hp\\Documents\\MGP\\CyberBullying Detection\\Dataset\\twitter_stemed_dataset.csv")

In [26]:
df.fillna('')

,index,id,Text,Annotation,oh_label,correct_text
0,5.74948705591165E+017,5.74948705591165E+017,i read them in change in meaning the history o...,none,0.0,read change mean history slavery
1,5.71917888690393E+017,5.71917888690393E+017,shreyabafna3 now you claim that people who tri...,none,0.0,shreyabafna3 claim people try stop become terr...
2,3.90255841338601E+017,3.90255841338601E+017,call me but when i go to an auto place id rath...,sexism,1.0,call go auto place id rather talk guy
3,5.68208850655916E+017,5.68208850655916E+017,g0ssipsquirrelx wrong the example of and the e...,racism,1.0,g0ssipsquirrelx wrong example exactly
4,5.75596338802373E+017,5.75596338802373E+017,no no no no no no,none,0.0,
...,...,...,...,...,...,...
16846,5.75606766236475E+017,5.75606766236475E+017,feeling so sorry for the they should be safe a...,none,0.0,feel sorry safe at go home
16847,5.72333822886326E+017,5.72333822886326E+017,pretty good were happy with well never eating ...,none,0.0,pretty good happy well never eat place
16848,5.72326950057845E+017,5.72326950057845E+017,lemon we please go just one season of without ...,none,0.0,lemon pleas go one season without someone call...
16849,5.74799612642357E+017,5.74799612642357E+017,you are too stupid to talk to blocked,none,0.0,stupid talk block


In [27]:
from sklearn.model_selection import train_test_split
X=df.drop("oh_label",axis=1)
Y=df["oh_label"]
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=0.3,random_state=10)

In [28]:
train_x.fillna(" ", axis=0, inplace=True)
test_x.fillna(" ", axis=0, inplace=True)
train_y.fillna(0.0, inplace=True)
test_y.fillna(0, inplace=True)

In [29]:
train_x.isna().sum()

index           0
id              0
Text            0
Annotation      0
correct_text    0
dtype: int64

In [35]:
type(train_x["Text"].iloc[2])

str

In [30]:
class Word_to_Vect():
    '''Function that returns word embedding, if passed list of sentences and size of vector'''
    
    def __init__(self):
        pass
    
    def w2v(self, corpus, size):
        
        #tokenization and model preparation
        tokenize_sent = [str(sent).split() for sent in corpus]
        #creating vord2vec for every word in every sentence in corpus
        self.w2v = word2vec.Word2Vec(sentences=tokenize_sent,vector_size=size, min_count=1)
        
        return self.w2v
    
    def transform(self, X_corpus, size):
        
        array_wordEmbed = []
        for sent in X_corpus:
            vec = np.zeros(size).reshape((1, size))
            count = 0.
            if sent == '':
                a = vec
            else:
                sent = sent.split(' ')
                for word in sent:
                    vec += w2v[word].reshape((1,size))
                    count +=1
            if count !=0:
                vec /= count
            a = vec
            array_wordEmbed.append(a)
        return np.concatenate(tuple(array_wordEmbed))

In [32]:
w2v = Word_to_Vect().w2v(corpus = [sentence for sentence in X['correct_text']], size = len(X['Text']))

In [35]:
w2v.wv.similar_by_vector('fast')


[('may', 0.9999833703041077),
 ('woman', 0.9999833106994629),
 ('lot', 0.9999832510948181),
 ('made', 0.9999831318855286),
 ('little', 0.9999831318855286),
 ('use', 0.9999830722808838),
 ('still', 0.999983012676239),
 ('thing', 0.999983012676239),
 ('away', 0.999983012676239),
 ('must', 0.999983012676239)]

In [36]:
X_w2v = pd.DataFrame({word:w2v.wv[word] for sent in X['correct_text'] for word in str(sent).split()})

In [ ]:
df_new=pd.concat([X_w2v,])

In [37]:
x_train2, x_test2, y_train2,y_test2 = train_test_split(X_w2v, Y, test_size=0.33, random_state=101)

In [38]:
x_train2

,read,change,mean,history,slavery,shreyabafna3,claim,people,try,stop,...,jcsliva1999,pott,prosecution,lorry,overdramat,feat,apocalypse,rhoward617,rossbarnes9,msaleh14
5157,-0.014552,-0.006437,-0.012667,-0.009042,-0.009258,-0.000313,-0.007765,-0.015689,-0.016307,-0.013464,...,-1.350693e-05,-1.781613e-04,-0.000338,-0.000341,-0.000169,-0.000561,-0.000374,-0.000127,-0.000136,-0.000048
16382,0.018108,0.007978,0.015795,0.011227,0.011521,0.000415,0.009580,0.019574,0.020443,0.016867,...,4.715468e-05,1.461526e-04,0.000369,0.000475,0.000125,0.000663,0.000507,0.000082,0.000126,-0.000001
362,0.012071,0.005271,0.010449,0.007470,0.007717,0.000257,0.006364,0.013011,0.013608,0.011187,...,-6.607515e-07,7.989909e-05,0.000232,0.000353,0.000089,0.000403,0.000311,0.000113,0.000105,-0.000014
9322,-0.001397,-0.000648,-0.001196,-0.000868,-0.000877,-0.000053,-0.000702,-0.001556,-0.001608,-0.001329,...,5.441904e-05,-4.545852e-05,-0.000044,-0.000010,0.000006,-0.000084,-0.000021,-0.000053,-0.000064,0.000007
15520,0.020351,0.008917,0.017543,0.012593,0.012915,0.000344,0.010761,0.021843,0.022883,0.018831,...,3.204857e-05,2.499920e-04,0.000415,0.000512,0.000149,0.000753,0.000529,0.000125,0.000190,0.000046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,-0.005108,-0.002254,-0.004411,-0.003215,-0.003288,-0.000114,-0.002673,-0.005373,-0.005673,-0.004675,...,8.475355e-06,4.553855e-07,-0.000044,-0.000126,-0.000023,-0.000146,-0.000175,0.000017,-0.000006,0.000041
12363,-0.017914,-0.007885,-0.015515,-0.011183,-0.011458,-0.000337,-0.009571,-0.019278,-0.020145,-0.016660,...,-5.169834e-05,-1.923576e-04,-0.000373,-0.000474,-0.000117,-0.000597,-0.000487,-0.000071,-0.000120,-0.000042
5695,0.045674,0.020085,0.039550,0.028266,0.029183,0.000860,0.024286,0.049073,0.051424,0.042335,...,1.211772e-04,5.108151e-04,0.000909,0.001222,0.000420,0.001600,0.001160,0.000278,0.000429,0.000060
8006,0.035466,0.015550,0.030751,0.022006,0.022622,0.000677,0.018803,0.038155,0.039792,0.032859,...,1.330438e-04,3.386387e-04,0.000737,0.000916,0.000373,0.001230,0.000889,0.000242,0.000274,0.000065


In [39]:
print(Y.shape)
print(X.shape)

(16851,)
(16851, 5)


In [ ]:
from xgboost import XGBClassifier
clf=XGBClassifier()
clf.fit(x_train2,y_train2)

C:\Users\hp\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:03:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
